In [15]:
# The second and third cells works on most machines. The first and last cells works on windows only
# Run the cell in order
# A GUI will pop up after running the 4th cell, close the GUI to get the next schedule

import tkinter as tk
from tkinter import ttk
from tkinter import Button
import time
from PIL import ImageGrab
from threading import Thread
from time import sleep


def getter(no):
    print("Not done!")
    sleep(1)
    x=30
    y=35
    x1=1920
    y1=1033
    ImageGrab.grab(include_layered_windows=False).crop((x,y,x1,y1)).save("C:\\Users\\giatl\\OneDrive\\Desktop\\scheds\\scheds"  + str(no) + ".png")
    print("Done!")


def setupWidget(widget1, window):
    days_in_week = [ "Monday", "Tuesday", "Wednesday", "Thursday", "Friday","Saturday"]
    times = []
    for i in range(8, 17):
        for j in range(6):
            for k in [0,5]:
                if i == 16:
                    if j >3:
                        continue
                times.append(str(i) + ":" + str(j) + str(k))

    for i in range(5):
        widget1.append(Button(window, text = days_in_week[i],fg ='black', bg = "grey"))
        widget1[-1].grid(row = 0, column=1 + i*4, sticky = 'nsew', columnspan= 4)
    for i in range(1,len(times),3):
        widget1.append(ttk.Label(window, text = times[i-1]))
        widget1[-1].grid(row = i, column=0, rowspan = 3)

def assignWidget(widget1, day_period, offset, window):
    onehr = int(50/5)
    one_half_hr = int(75/5)
    twohr = int(120/5)
    threehrs = int(180/5)

    span = {}
    labe = {}
    for c in course1:
        span[str(c)] = onehr
        labe[str(c)] = "C50"
    for c in course2:
        span[str(c)] = one_half_hr
        labe[str(c)] = "C75"
    for c in course3:
        span[str(c)] = threehrs
        labe[str(c)] = "Lab"
    for c in course4:
        span[str(c)] = one_half_hr
        labe[str(c)] = "1.5hr common hr: "+ str(c) 
    for c in course5:
        span[str(c)] = onehr
        labe[str(c)] = "Lang"
    for c in course6:
        span[str(c)] = onehr
        labe[str(c)] = "Lang"
    for c in course7:
        span[str(c)] = onehr
        labe[str(c)] = "1h common hr"
    for c in course8:
        span[str(c)] = twohr
        labe[str(c)] = "Faculty Meeting (2hr)"
    for c in course9:
        span[str(c)] = threehrs
        labe[str(c)] = "Seminar"
    
    color = {
        "0" : "#fe4a49",
        "1" : "#651e3e",
        "2" : "#00b159",
        "3" : "#ff3377",
        "4" : "#55a61c",
        "5" : "#6497b1",
        "6" : "#3d1e6d",
        "7" : "#451e3e",
        "8" : "#0392cf",
        "9" : "#dbaa00",
        "10" : "#0054ff",
        "11" : "#fe8a71",
        "12" : "#0e9aa7",
        "13" : "#081f2d",
        "14" : "#d26939",
        "15" : "#195465",
        "16" : "#673888",
        "17" : "#7bc043",
        "18" : "#89ce00",
        "19" : "#854442",
        "20" : "#fc6716",
        "21" : "#ff8948",
        "22" : "#9bd21d",
        "23" : "#ad89ff",
        "24" : "#6a329f",
    }
    spanall = list(course4)
    spanall.extend(list(course7))
    spanall.extend(list(course8))
    for r in weekdays:
        for c in periods:
            if day_period[r,c] != " __ ":
                course = day_period[r,c]
                if int(course) in spanall:
                    widget1.append(Button(window, text = labe[course],fg ='white', bg=color[course]))
                    widget1[-1].grid(row = 1+c*6, column=1 + offset +r*4 - 3, sticky = 'nsew', rowspan= span[course], columnspan = 4)
                    continue
                widget1.append(Button(window, text = labe[course] +": "+ course,fg ='white', bg=color[course]))
                widget1[-1].grid(row = 1+c*6, column=1 + offset +r*4, sticky = 'nsew', rowspan= span[course])


def outputGUI(day_period1, day_period2, day_period3, day_period4,no):
    thread = Thread(target = getter, args = [no])
    # New threads for Image grab
    # window
    window = tk.Tk()
    window.title('Grid')
    window['bg'] = 'white'
    window.geometry('1920x1080')

    # widgets
    widget1 = []

    # widget[1] = Button(window, text= 'Label2', background= 'blue', fg="orange")


    # define a grid
    window.columnconfigure(list(range(1+5*4)), weight=1, uniform='a')
    # window.columnconfigure(3, weight =2, uniform='a')
    window.rowconfigure(list(range(1+(num_periods+2)*6)), weight = 1, uniform = 'a')
    window.rowconfigure(0, weight = 5, uniform = 'a')
    #window.rowconfigure(3, weight=3, uniform='a')
    assignWidget(widget1,day_period1, 0, window)
    assignWidget(widget1,day_period2, 1, window)
    assignWidget(widget1,day_period3, 2, window)
    assignWidget(widget1,day_period4, 3, window)
    setupWidget(widget1, window)

    # place a widget

    #widget[1].grid(row = 1, column= 1, rowspan=3, columnspan= 2 , sticky = 'nsew')



    #run
    # thread.start()
    # window.after(2000, window.destroy)
    
    window.mainloop()
    # thread.join()




In [13]:
# To be able to run this code, to install numpy, scipy, gurobipy (you need to install gurobi and optain a license to run this model)

import numpy as np
import gurobipy as gp
from gurobipy import GRB

def print2D(day_period):
    for r in day_period:
        for c in r:
            if c == " __ ":
                print(c, end="")
            elif len(c) == 1:
                print("  "+c+" ", end="")
            else:
                print(" "+c+" ", end="")

        print("")

try:
    switch = 1
    solutionNum = 30
    # Create a new model
    m = gp.Model("matrix1")

    M = 10^6
    num_days = 6
    days = range(num_days)
    last_day = 5
    weekdays = range(last_day)
    num_courses = 25
    last_course1 = 6
    last_course2 = 15     # 10
    last_course3 = last_course2+1     # lab
    last_course4 = last_course3+1 #12     # common hour 
    last_course5 = last_course4+3 #15     # lang morning
    last_course6 = last_course5+1 #16     # lang afternoon
    last_course7 = last_course6+1 #17      # commonhr 1h
    last_course8 = last_course7+1 #18     # faculty meeting
    last_course9 = last_course8+1 #19     # seminar
    courses = range(num_courses) 
    course1 = range(last_course1 + 1)
    course2 = range(last_course1 + 1, last_course2 + 1)
    course3 = range(last_course2 +1 , last_course3 + 1)
    course4 = range(last_course3 +1, last_course4 + 1)
    course5 = range(last_course4 +1, last_course5 + 1)
    course6 = range(last_course5 + 1, last_course6 +1)
    course7 = range(last_course6 + 1, last_course7 +1)
    course8 = range(last_course7 + 1, last_course8 +1)
    course9 = range(last_course8 + 1, last_course9 +1)

    num_weekdays = 5
    early_starts = range(num_weekdays)

    num_periods = 16
    last_aft_prd = 15   # last afternoon period
    last_mor_prd = 7   # last morning period
    seminar_periods = range(last_mor_prd+1,last_mor_prd+5)
    common_hour_period = range(num_periods - 1)
    lab_periods = range(last_mor_prd+1,last_mor_prd+5)
    faculty_period = range(last_aft_prd -2)
    periods = range(num_periods)
    aft_prd = range(last_mor_prd + 1 ,last_aft_prd+1)
    mor_prd = range(last_mor_prd + 1)
    dec_var_num = num_days*num_courses*num_periods + num_courses*num_periods + num_weekdays + last_course2 + 1
    reach_course_avail = num_days*num_courses*num_periods + num_courses*num_periods + num_weekdays
    dpc_offset = num_days*num_courses*num_periods
    dpcw_offset = num_days*num_courses*num_periods + num_courses*num_periods
    day_offset = num_courses*num_periods
    period_offset = num_courses
    # course_in_period = num_periods*num_courses


    # Special sets --------------------------------------------------------------------------------------
    # Course that meets five times a week
    five_prd_courses = list(course3)
    five_prd_courses.extend(list(course5))
    five_prd_courses.extend(list(course6))

    # periods that are not valid for falcuty meeting
    non_falc_periods = list(periods)
    for p in faculty_period:
        non_falc_periods.remove(p)

    # periods that are not valid for seminar meeting
    non_sem_periods = list(periods)
    for p in seminar_periods:
        non_sem_periods.remove(p)

    # Courses that meets for 1.5 hour
    one_and_half_hr_courses = list(course2)
    one_and_half_hr_courses.extend(list(course4))

    # Courses that only start in the afternoon
    afternoon_courses = list(course3)
    afternoon_courses.extend(list(course6))

    # Start time that are invalid for labs
    non_lab_periods = list(periods)
    for l in lab_periods:
        non_lab_periods.remove(l)
    
    # Courses, each of which meets five times a week at the same start time
    five_prd_courses_fixed = five_prd_courses.copy()
    for c3 in course3:
        five_prd_courses_fixed.remove(c3)

    #Language courses
    lang_courses = list(course5)
    lang_courses.extend(list(course6))

    # Set of morning language courses and labs
    mlang_lab_courses = list(course5)
    mlang_lab_courses.extend(list(course3))

    # Courses/common hour that meets for 1 hour    
    one_hour_courses = list(course1)
    one_hour_courses.extend(list(course5))
    one_hour_courses.extend(list(course6))
    one_hour_courses.extend(list(course7))

    # Courses that meets for 2 hours
    two_hour_courses = list(course8)

    # Courses that meets for 3 hours
    three_hour_courses = list(course3)
    three_hour_courses.extend(course9)
    
    # courses that are not 1.5hr common hour
    not_one_and_half_comhr_courses = list(courses)
    for c4 in course4:
        not_one_and_half_comhr_courses.remove(c4)
    
    # Set of courses that meets for 1hr except for 1hr common hr
    one_hour_courses_exclude_common_hr = list(course1)
    one_hour_courses_exclude_common_hr.extend(list(course5))
    one_hour_courses_exclude_common_hr.extend(list(course6))

    # Courses that are not 1hr common hr course
    not_one_comhr_courses = list(courses)
    for c7 in course7:
        not_one_comhr_courses.remove(c7)

    # Courses that are not faculty meeting
    not_falc_courses = list(courses)
    for c8 in course8:
        not_falc_courses.remove(c8)

    # Courses that meets for 1hr excluding afternoon language class
    one_hour_courses_not_aft_lang = list(course1)
    one_hour_courses_not_aft_lang.extend(list(course5))
    one_hour_courses_not_aft_lang.extend(list(course7))

    # Courses that are neither seminar nor labs nor afternoon language course
    not_sem_lab_aft_lang_courses = list(courses)
    for c9 in course9:
        not_sem_lab_aft_lang_courses.remove(c9)
    for c3 in course3:
        not_sem_lab_aft_lang_courses.remove(c3) 
    for c6 in course6:
        not_sem_lab_aft_lang_courses.remove(c6)   

    # Special sets --------------------------------------------------------------------------------------


    default_mat_row = 7000

    x = m.addMVar(shape=dec_var_num, vtype=GRB.BINARY, name="x")

    # Set objective
    obj = np.zeros((dec_var_num))
    # obj 1: 1.5 common hr
    for c4 in course4:
        for d in weekdays:
            for p in common_hour_period:
                obj[d*day_offset + p*period_offset + c4] = 20
    
    for c4 in course4:
        for d in weekdays:
            for p in range(5,14):
                obj[d*day_offset + p*period_offset + c4] = 60
    
    # obj 2: 1 common hr
    for c7 in course7:
        for d in weekdays:
            for p in periods:
                obj[d*day_offset + p*period_offset + c7] = 3 

    for c7 in course7:
        for d in weekdays:
            for p in range(5,14):
                obj[d*day_offset + p*period_offset + c7] = 8

    # standard_courses = list(course1)
    # standard_courses.extend(list(course2))
    for c2 in course2:
        for d in weekdays:
            for p in periods:
        # obj[reach_course_avail + c] = 50
                obj[d*day_offset + p*period_offset + c2] = 30            
    
    # # obj 3: faculty meeting
    # for c8 in course8:
    #     for d in weekdays:
    #         for p in faculty_period:
    #             obj[d*day_offset + p*period_offset + c8] = 1 

    
    m.setObjective(obj @ x, GRB.MAXIMIZE)

    A = np.zeros((default_mat_row,dec_var_num))
    rhs = []
    cnst = 0

   
    # (I) Number of meetings in specific courses ----------------------------------------------------------------------

    #const 2 coeficients =. 1hr standard meets three times per week
    for c1 in course1:
        for d in days:
            for p in periods:
                A[cnst, d*day_offset + p*period_offset + c1] = 1
        
        A[cnst, reach_course_avail + c1] = -3
        cnst += 1
        rhs.append(0)
        
    #const 3 coeficients =. 1.5 hr standard class meets twice per week
    for c2 in course2:
        for d in days:
            for p in periods:
                A[cnst, d*day_offset + p*period_offset + c2] = 1
        A[cnst, reach_course_avail + c2] = -2
        cnst += 1
        rhs.append(0)
    
    #const 3.1 coeficients = (5 periods for several classes)
    # print(five_prd_courses)
    for c3 in five_prd_courses:
        for d in days:
            for p in periods:
                A[cnst, d*day_offset + p*period_offset + c3] = 1
        cnst += 1
        rhs.append(5)

    # temp sem enforcer
    for c9 in course9:
        for d in weekdays:
            for p in seminar_periods:
                A[cnst, d*day_offset + p*period_offset + c9] = 1
    cnst+=1
    rhs.append(1)

    # (I) Number of meetings in specific courses ----------------------------------------------------------------------


    # (II) Invalid course periods ----------------------------------------------------------------------

    #const 1 coeficients =. No classes on Saturday
    for p in periods:
        for c in courses:
            A[cnst,5*day_offset + p*period_offset + c] = 1
    cnst += 1   
    rhs.append(0)    

    # non_falc periods
    for p in non_falc_periods:
        for d in weekdays:
            for c8 in course8:
                A[cnst, d*day_offset + p*period_offset + c8] = 1
    cnst+= 1
    rhs.append(0)

    # non_sem periods
    for p in non_sem_periods:
        for d in weekdays:
            for c9 in course9:
                A[cnst, d*day_offset + p*period_offset + c9] = 1
    cnst+= 1
    rhs.append(0)


    # const 13  coeficients <=. Start time 15 is not valid for 1.5hr courses
    # print(one_and_half_hr_courses)
    for d in weekdays:
        for c2 in one_and_half_hr_courses:
            A[cnst, d*day_offset + (num_periods-1)*period_offset + c2] = 1
    cnst +=1
    rhs.append(0)
    
    # const 15 coefs - no afternoon meetings for 3 lang courses

    for c5 in course5:
        for d in weekdays:
            for ap in aft_prd:
                A[cnst, d*day_offset + ap*period_offset + c5] = 1
    cnst += 1
    rhs.append(0)

    # const 16 coefs - no morning meetings for labs and afternoon lang course
    for ac in afternoon_courses:
        for d in weekdays:
            for mp in mor_prd:
                A[cnst, d*day_offset + mp*period_offset + ac] = 1
    cnst += 1
    rhs.append(0)

    #const 17 coefs - lab meetings should not be in the morning or late afternoons

    for nlp in non_lab_periods:
        for d in weekdays:
            for c3 in course3:
                A[cnst, d*day_offset + nlp*period_offset + c3] = 1
    cnst += 1
    rhs.append(0)

    
    
    # (II) Invalid course periods ----------------------------------------------------------------------



    # (V) Fixed meeting times ----------------------------------------------------------------------

    # const 6 coeficients =. All 1hr standard course will be schedule at the same start time
    for c1 in course1:
        for p in periods:
            for d in days:
                A[cnst, d*day_offset + p*period_offset + c1] = 1
            A[cnst, dpc_offset + p*period_offset + c1] = -3
            cnst +=1
            rhs.append(0)
    # const 7 coeficients =. All 1.5hr standard course will be schedule at the same start time
    for c2 in course2:
        for p in periods:
            for d in days:
                A[cnst, d*day_offset + p*period_offset + c2] = 1
            A[cnst, dpc_offset + p*period_offset + c2] = -2
            cnst +=1
            rhs.append(0)
    
    # const 7.1 coeficients. All language courses will be schedule at the same start time
    
    for fpcf in five_prd_courses_fixed:
        for p in periods:
            for d in days:
                A[cnst, d*day_offset + p*period_offset + fpcf] = 1
            A[cnst, dpc_offset + p*period_offset + fpcf] = -5
            cnst +=1
            rhs.append(0)
    
    # (V) Fixed meeting times ----------------------------------------------------------------------

    

    A=np.delete(A, range(cnst,default_mat_row), axis=0) 
    # print(np.shape(A))
    # print(len(rhs))

    nprhs = np.array(rhs)
    # print(nprhs)
    m.addConstr(A @ x == nprhs, name="ceq")  

    A = np.zeros((default_mat_row,dec_var_num))
    rhs = []
    cnst = 0

    # (VII) Standard course available ----------------------------------------------------------------------
    standard_courses = list(course1)
    standard_courses.extend(list(course2))
    for c in standard_courses:
        A[cnst, reach_course_avail + c] =-1
    cnst += 1
    rhs.append(-11)

    for c in course1:
        A[cnst, reach_course_avail + c] =-1
    cnst += 1
    rhs.append(-4)

    for c in course2:
        A[cnst, reach_course_avail + c] =-1
    cnst += 1
    rhs.append(-5)

    for d in weekdays:
        for p in common_hour_period:
            for c in course4:
                A[cnst, d*day_offset + p*period_offset + c] =-1
    cnst += 1
    rhs.append(-2)
    # (VII) Standard course available ----------------------------------------------------------------------



    # (VIII) Early starts ----------------------------------------------------------------------

    # const ES - at most two days can have early start out of the weekdays (lang courses are not allowed to start early)
    for ea in early_starts:
        A[cnst, dpcw_offset + ea] = 1
    cnst += 1
    rhs.append(2)

    for ea in early_starts:
        for c in courses:
            A[cnst, ea*day_offset + c] = 1
        A[cnst, dpcw_offset + ea] = -M
        cnst += 1
        rhs.append(0)
    # (VIII) Early starts ----------------------------------------------------------------------


    # (IV) Day spacing for each course ------------------------------------------------------------------------

    # const 4 coeficients <=. Spacing for 1hr standard course
    for c1 in course1:
        for d in weekdays:
            for p in periods:
                A[cnst, d*day_offset + p*period_offset + c1] = 1
                A[cnst, ((d+1)%num_days)*day_offset + p*period_offset + c1] = 1
                A[cnst, ((d+2)%num_days)*day_offset + p*period_offset + c1] = 1
            cnst += 1
            rhs.append(2)
            # rhs.append(1)


    # const 5 coeficients <=. Day spacing for 1.5hr standard course
    for c2 in course2:
        for d in weekdays:
            for p in periods:
                A[cnst, d*day_offset + p*period_offset + c2] = 1
                A[cnst, ((d+1)%(num_days-1))*day_offset + p*period_offset + c2] = 1
            cnst += 1
            rhs.append(1)

    # const 5.1. Day spacing for 5 meetings courses
    for c2 in five_prd_courses:
        for d in weekdays:
            for p in periods:
                A[cnst, d*day_offset + p*period_offset + c2] = 1
            cnst += 1
            rhs.append(1)

    # (IV) Day spacing for each course ------------------------------------------------------------------------



    # (VI) Collision of courses' meetings -------------------------------------------------------------------------

    # const 4.5. 1hr standard courses do not collide
    for d in weekdays:
        for p in periods:
            for c1 in course1:
                A[cnst, d*day_offset + p*period_offset + c1] = 1
                if p+1 <= num_periods -1:
                    A[cnst, d*day_offset + (p+1)*period_offset + c1] = 1
            cnst += 1
            rhs.append(1)

    # const 5.5. 1.5hr courses do not collide
    for d in weekdays:
        for p in range(num_periods-2):
            for coah in one_and_half_hr_courses:
                A[cnst, d*day_offset + p*period_offset + coah] = 1
                A[cnst, d*day_offset + (p+1)*period_offset + coah] = 1
                A[cnst, d*day_offset + (p+2)*period_offset + coah] = 1
            cnst += 1
            rhs.append(1)

    # const 4.6. 1hr common hours do not collide
    for d in weekdays:
        for p in periods:
            for c7 in course7:
                A[cnst, d*day_offset + p*period_offset + c7] = 1
                if p+1 <= num_periods -1:
                    A[cnst, d*day_offset + (p+1)*period_offset + c7] = 1
            cnst += 1
            rhs.append(1)

    #const 5.6. Mor & Aft lang courses cannot collide
    for p in mor_prd:
        for ml in lang_courses:
            A[cnst, p*period_offset + ml] = 1
            A[cnst, (p+1)*period_offset + ml] = 1
        cnst +=1
        rhs.append(1)
    
    # const 5.7 (mlang + lab cant conflict). Works at the moment because labs are only scheduled for the afternoon. In other words with the current model a lab always starts after morning language classes.
    for p in mor_prd:
        for mlc in mlang_lab_courses:
            A[cnst, p*period_offset + mlc] = 1
            A[cnst, (p+1)*period_offset + mlc] = 1
        cnst +=1
        rhs.append(1)
    
    # non-colide for 1hr courses with 1.5hr course
    for c1 in course1:
        for p in periods:
            for d in weekdays:
                for c2 in course2:
                    if (p-1 >= 0):
                        A[cnst, d*day_offset + (p-1)*period_offset + c2] = 1
                    if (p-2 >= 0):
                        A[cnst, d*day_offset + (p-2)*period_offset + c2] = 1
                    A[cnst, d*day_offset + p*period_offset + c2] = 1
                    if (p+1 <= num_periods):
                        A[cnst, d*day_offset + (p+1)*period_offset + c2] = 1
                    A[cnst, d*day_offset + p*period_offset + c1] = M 
                    cnst+=1
                    rhs.append(M)

    # (VI) Collision of courses' meetings -------------------------------------------------------------------------

    
    # (III) Special arangement for courses ---------------------------------------------------------------------
    # const 18 coefs - no classes overlap with 1.5 common hours 
    for ch in course4:
        for d in days:
            for p in common_hour_period:
                for ohc in one_hour_courses:
                    if (p-1 >= 0):
                        A[cnst, d*day_offset + (p-1)*period_offset + ohc] = 1
                for c2 in course2:
                    if (p-1 >= 0):
                        A[cnst, d*day_offset + (p-1)*period_offset + c2] = 1
                    if (p-2 >= 0):
                        A[cnst, d*day_offset + (p-2)*period_offset + c2] = 1
                for twohc in two_hour_courses:
                    for index in range(1,4):
                        if (p - index >= 0):
                            A[cnst, d*day_offset + (p-index)*period_offset + twohc] = 1
                for thc in three_hour_courses:
                    for index in range(1,6):
                        if (p - index >= 0):
                            A[cnst, d*day_offset + (p-index)*period_offset + thc] = 1
                for cl in not_one_and_half_comhr_courses:
                    A[cnst, d*day_offset + p*period_offset + cl] = 1
                    A[cnst, d*day_offset + (p+1)*period_offset + cl] = 1 
                    if (p+2) <= last_aft_prd : 
                        A[cnst, d*day_offset + (p+2)*period_offset + cl] = 1 
                        
                A[cnst, d*day_offset + p*period_offset + ch] = M 
                cnst+=1
                rhs.append(M)
    
    # const 19 coefs - no classes overlap with 1hr common hours 
    for ch in course7:
        for d in days:
            for p in periods:
                for ohc in one_hour_courses_exclude_common_hr:
                    if (p-1 >= 0):
                        A[cnst, d*day_offset + (p-1)*period_offset + ohc] = 1
                for c2 in one_and_half_hr_courses:
                    if (p-1 >= 0):
                        A[cnst, d*day_offset + (p-1)*period_offset + c2] = 1
                    if (p-2 >= 0):
                        A[cnst, d*day_offset + (p-2)*period_offset + c2] = 1
                for twohc in two_hour_courses:
                    for index in range(1,4):
                        if (p - index >= 0):
                            A[cnst, d*day_offset + (p-index)*period_offset + twohc] = 1
                for thc in three_hour_courses:
                    for index in range(1,6):
                        if (p - index >= 0):
                            A[cnst, d*day_offset + (p-index)*period_offset + thc] = 1
                for cl in not_one_comhr_courses:
                    A[cnst, d*day_offset + p*period_offset + cl] = 1
                    if (p+1) <= last_aft_prd : 
                        A[cnst, d*day_offset + (p+1)*period_offset + cl] = 1 
                        
                A[cnst, d*day_offset + p*period_offset + ch] = M 
                cnst+=1
                rhs.append(M)

    # const 20 coefs - no classes overlap with 2h faculty meeting  
    for ch in course8:
        for d in days:
            for p in faculty_period:
                for ohc in one_hour_courses:
                    if (p-1 >= 0):
                        A[cnst, d*day_offset + (p-1)*period_offset + ohc] = 1
                for c2 in one_and_half_hr_courses:
                    if (p-1 >= 0):
                        A[cnst, d*day_offset + (p-1)*period_offset + c2] = 1
                    if (p-2 >= 0):
                        A[cnst, d*day_offset + (p-2)*period_offset + c2] = 1
                for thc in three_hour_courses:
                    for index in range(1,6):
                        if (p - index >= 0):
                            A[cnst, d*day_offset + (p-index)*period_offset + thc] = 1
                for cl in not_falc_courses:
                    A[cnst, d*day_offset + p*period_offset + cl] = 1
                    for index in range(1,4):
                        if (p+index) <= num_periods-1 : 
                            A[cnst, d*day_offset + (p+index)*period_offset + cl] = 1 
                A[cnst, d*day_offset + p*period_offset + ch] = M 
                cnst+=1
                rhs.append(M)


    # const 21 coefs - no classes overlap with seminar except for labs, and afternoon language course  
    for ch in course9:
        for d in days:
            for p in faculty_period:
                for ohc in one_hour_courses_not_aft_lang:
                    if (p-1 >= 0):
                        A[cnst, d*day_offset + (p-1)*period_offset + ohc] = 1
                for c2 in one_and_half_hr_courses:
                    if (p-1 >= 0):
                        A[cnst, d*day_offset + (p-1)*period_offset + c2] = 1
                    if (p-2 >= 0):
                        A[cnst, d*day_offset + (p-2)*period_offset + c2] = 1
                for cl in not_sem_lab_aft_lang_courses:
                    A[cnst, d*day_offset + p*period_offset + cl] = 1
                    for index in range(1,6):
                        if (p+index) <= num_periods-1 : 
                            A[cnst, d*day_offset + (p+index)*period_offset + cl] = 1 
                A[cnst, d*day_offset + p*period_offset + ch] = M 
                cnst+=1
                rhs.append(M)
    
    # (III) Special arangement for courses ---------------------------------------------------------------------

    A=np.delete(A, range(cnst,default_mat_row), axis=0) 
    print(np.shape(A))
    #print(len(rhs))

    nprhs = np.array(rhs)    
    #print(nprhs)

    m.addConstr(A @ x <= nprhs, name="cleq")
    
    if switch == 1:
        # m.setParam("poolGap", 0.2) 
        m.setParam("PoolSolutions",solutionNum)
        m.setParam("PoolSearchMode", 2)

 # Optimize model
    m.optimize()
    
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Encountered an attribute error')

(5755, 2821)
Set parameter PoolSolutions to value 30
Set parameter PoolSearchMode to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 7 PRO 7840U w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6104 rows, 2821 columns and 78563 nonzeros
Model fingerprint: 0x2ffee3ab
Variable types: 0 continuous, 2821 integer (2821 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [3e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 4239 rows and 942 columns
Presolve time: 0.27s
Presolved: 1865 rows, 1879 columns, 53366 nonzeros
Variable types: 0 continuous, 1879 integer (1879 binary)

Root relaxation: objective 1.110847e+03, 894 iterations, 0.04 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth

In [16]:
if switch == 1:
        #print(x.X[4*day_offset: 5*day_offset])
        for index in range(0, solutionNum, int(solutionNum/5)):
            m.setParam("solutionNumber", index)
            # m.setParam("PoolObjVal", index)
            ans = x.Xn

            # ans = x.X
            print(x.Xn)
            
            total = 0
            empty = " __ "
            day_period1 = np.full((num_days, num_periods),empty)
            day_period2 = np.full((num_days, num_periods),empty)
            day_period3 = np.full((num_days, num_periods),empty)
            day_period4 = np.full((num_days, num_periods),empty)

            for c1 in course1:
                for d in days:
                    for p in periods:
                        if ans[d*day_offset + p*period_offset + c1] == 1:
                            if day_period1[d,p] == empty :
                                # A[cnst, d*day_offset + (p+2)*period_offset + c] = 1
                                #print("scheduled in day ",d, "at period ",p, ": course ", c1 )
                                day_period1[d,p] = c1

            for c2 in course2:
                for d in days:
                    for p in periods:
                        if ans[d*day_offset + p*period_offset + c2] == 1:
                            if day_period2[d,p] == empty :
                                # A[cnst, d*day_offset + (p+2)*period_offset + c] = 1
                                #print("scheduled in day ",d, "at period ",p, ": course ", c2 )
                                day_period2[d,p] = c2
            
            for c3 in course3:
                for d in days:
                    for p in periods:
                        if ans[d*day_offset + p*period_offset + c3] == 1:
                            if day_period3[d,p] == empty :
                                # A[cnst, d*day_offset + (p+2)*period_offset + c] = 1
                                # print("scheduled in day ",d, "at period ",p, ": course ", c3 )
                                day_period3[d,p] = c3

            non_standardlab_courses = list(course4)
            non_standardlab_courses.extend(list(course5))
            non_standardlab_courses.extend(list(course6))
            non_standardlab_courses.extend(list(course7))
            non_standardlab_courses.extend(list(course8))
            non_standardlab_courses.extend(list(course9))
            for nsc in non_standardlab_courses:
                for d in days:
                    for p in periods:
                        if ans[d*day_offset + p*period_offset + nsc] == 1:
                            if day_period4[d,p] == empty :
                                # A[cnst, d*day_offset + (p+2)*period_offset + c] = 1
                                # print("scheduled in day ",d, "at period ",p, ": course ", nsc )
                                day_period4[d,p] = nsc
            
            print("-----------------------------------------------------------")
            print2D(day_period1)
            # print(day_period1)
            print("-------------------------------------------------------------")
            print2D(day_period2)
            print("-------------------------------------------------------------")
            print2D(day_period3)
            print("-------------------------------------------------------------")
            print2D(day_period4)
            print("-------------------------------------------------------------")
            
            outputGUI(day_period1, day_period2, day_period3, day_period4, index)
            # for c in courses:
            #     for d in days:
            #         for p in periods:
            #             if ans[d*day_offset + p*period_offset + c] == 1:
            #                 print("course ", c, "scheduled in day ",d, "at period ",p)

            # for d in days:
            #     print("day ",d)
            #     for p in periods:
            #         print("period ", p, ":", end=" ")
            #         for c in courses:
            #             if ans[d*day_offset + p*period_offset + c] == 1:
            #                 print("course ", c, end="   ")
            #         print("")

            print('Obj: %g' % m.PoolObjVal)


ans = x.X

# ans = x.X
# print(x.X)

total = 0
empty = " __ "
day_period1 = np.full((num_days, num_periods),empty)
day_period2 = np.full((num_days, num_periods),empty)
day_period3 = np.full((num_days, num_periods),empty)
day_period4 = np.full((num_days, num_periods),empty)

for c1 in course1:
    for d in days:
        for p in periods:
            if ans[d*day_offset + p*period_offset + c1] == 1:
                if day_period1[d,p] == empty :
                    # A[cnst, d*day_offset + (p+2)*period_offset + c] = 1
                    #print("scheduled in day ",d, "at period ",p, ": course ", c1 )
                    day_period1[d,p] = c1

for c2 in course2:
    for d in days:
        for p in periods:
            if ans[d*day_offset + p*period_offset + c2] == 1:
                if day_period2[d,p] == empty :
                    # A[cnst, d*day_offset + (p+2)*period_offset + c] = 1
                    #print("scheduled in day ",d, "at period ",p, ": course ", c2 )
                    day_period2[d,p] = c2

for c3 in course3:
    for d in days:
        for p in periods:
            if ans[d*day_offset + p*period_offset + c3] == 1:
                if day_period3[d,p] == empty :
                    # A[cnst, d*day_offset + (p+2)*period_offset + c] = 1
                    # print("scheduled in day ",d, "at period ",p, ": course ", c3 )
                    day_period3[d,p] = c3

non_standardlab_courses = list(course4)
non_standardlab_courses.extend(list(course5))
non_standardlab_courses.extend(list(course6))
non_standardlab_courses.extend(list(course7))
non_standardlab_courses.extend(list(course8))
non_standardlab_courses.extend(list(course9))
for nsc in non_standardlab_courses:
    for d in days:
        for p in periods:
            if ans[d*day_offset + p*period_offset + nsc] == 1:
                if day_period4[d,p] == empty :
                    # A[cnst, d*day_offset + (p+2)*period_offset + c] = 1
                    # print("scheduled in day ",d, "at period ",p, ": course ", nsc )
                    day_period4[d,p] = nsc

print("-----------------------------------------------------------")
print2D(day_period1)
# print(day_period1)
print("-------------------------------------------------------------")
print2D(day_period2)
print("-------------------------------------------------------------")
print2D(day_period3)
print("-------------------------------------------------------------")
print2D(day_period4)
print("-------------------------------------------------------------")

[-0.  0. -0. ...  1.  1.  1.]
-----------------------------------------------------------
 __  __  __   3  __   5  __  __  __  __  __  __  __  __  __  __ 
 __   1  __  __  __  __  __  __  __  __  __  __  __  __  __   6 
 __   1  __   3  __   5  __  __  __  __  __  __  __  __  __  __ 
 __  __  __  __  __  __  __  __  __  __  __  __  __  __  __   6 
 __   1  __   3  __   5  __  __  __  __  __  __  __  __  __   6 
 __  __  __  __  __  __  __  __  __  __  __  __  __  __  __  __ 
-------------------------------------------------------------
  9  __  __  __  __  __  __  __  __  __  __  15  __  __  10  __ 
 __  __  __  14  __  __   7  __  __   8  __  __  13  __  __  __ 
 __  __  __  __  __  __  __  __  __  __  __  15  __  __  10  __ 
  9  __  __  14  __  __   7  __  __   8  __  __  13  __  __  __ 
 __  __  __  __  __  __  __  __  __  __  __  __  __  __  __  __ 
 __  __  __  __  __  __  __  __  __  __  __  __  __  __  __  __ 
-------------------------------------------------------------
 __  _

In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import Button
import time
from PIL import ImageGrab


def getter():
    x=0
    y=100
    x1=3360
    y1=1900
    ImageGrab.grab().crop((x,y,x1,y1)).save("/Users/havietnhat/Desktop/scheds/sched10.png")

def setupWidget():
    days_in_week = [ "Monday", "Tuesday", "Wednesday", "Thursday", "Friday","Saturday"]
    times = []
    for i in range(8, 17):
        for j in range(6):
            for k in [0,5]:
                # if i == 8:
                #     if j < 3:
                #         continue
                if i == 16:
                    if j >3:
                        continue
                times.append(str(i) + ":" + str(j) + str(k))

    for i in range(5):
        widget1.append(Button(window, text = days_in_week[i],fg ='black', bg = "grey"))
        widget1[-1].grid(row = 0, column=1 + i*4, sticky = 'nsew', columnspan= 4)
    for i in range(1,len(times),3):
        widget1.append(ttk.Label(window, text = times[i-1]))
        widget1[-1].grid(row = i, column=0, rowspan = 3)

def assignWidget(day_period, offset):
    onehr = int(50/5)
    one_half_hr = int(75/5)
    twohr = int(120/5)
    threehrs = int(180/5)
    span = {}
    labe = {}
    for c in course1:
        span[str(c)] = onehr
        labe[str(c)] = "C50"
    for c in course2:
        span[str(c)] = one_half_hr
        labe[str(c)] = "C75"
    for c in course3:
        span[str(c)] = threehrs
        labe[str(c)] = "Lab"
    for c in course4:
        span[str(c)] = one_half_hr
        labe[str(c)] = "1.5hr common hr: "+ str(c) 
    for c in course5:
        span[str(c)] = onehr
        labe[str(c)] = "Lang"
    for c in course6:
        span[str(c)] = onehr
        labe[str(c)] = "Lang"
    for c in course7:
        span[str(c)] = onehr
        labe[str(c)] = "1h common hr"
    for c in course8:
        span[str(c)] = twohr
        labe[str(c)] = "Faculty Meeting (2hr)"
    for c in course9:
        span[str(c)] = threehrs
        labe[str(c)] = "Seminar"
    
    color = {
        "0" : "#fe4a49",
        "1" : "#651e3e",
        "2" : "#00b159",
        "3" : "#ff3377",
        "4" : "#55a61c",
        "5" : "#6497b1",
        "6" : "#3d1e6d",
        "7" : "#451e3e",
        "8" : "#0392cf",
        "9" : "#dbaa00",
        "10" : "#0054ff",
        "11" : "#fe8a71",
        "12" : "#0e9aa7",
        "13" : "#081f2d",
        "14" : "#d26939",
        "15" : "#195465",
        "16" : "#673888",
        "17" : "#7bc043",
        "18" : "#89ce00",
        "19" : "#854442",
        "20" : "#fc6716",
        "21" : "#ff8948",
        "22" : "#9bd21d",
        "23" : "#ad89ff",
        "24" : "#6a329f",
    }
    spanall = list(course4)
    spanall.extend(list(course7))
    spanall.extend(list(course8))
    for r in weekdays:
        for c in periods:
            if day_period[r,c] != " __ ":
                course = day_period[r,c]
                if int(course) in spanall:
                    widget1.append(Button(window, text = labe[course],fg ='white', bg=color[course]))
                    widget1[-1].grid(row = 1+c*6, column=1 + offset +r*4 - 3, sticky = 'nsew', rowspan= span[course], columnspan = 4)
                    continue
                widget1.append(Button(window, text = labe[course] +": "+ course,fg ='white', bg=color[course]))
                widget1[-1].grid(row = 1+c*6, column=1 + offset +r*4, sticky = 'nsew', rowspan= span[course])



# window
window = tk.Tk()
window.title('Grid')
window['bg'] = 'white'
window.geometry('1920x1080')

# widgets
widget1 = []


# define a grid
window.columnconfigure(list(range(1+5*4)), weight=1, uniform='a')
window.rowconfigure(list(range(1+(num_periods+2)*6)), weight = 1, uniform = 'a')
window.rowconfigure(0, weight = 5, uniform = 'a')

assignWidget(day_period1, 0)
assignWidget(day_period2, 1)
assignWidget(day_period3, 2)
assignWidget(day_period4, 3)
setupWidget()

# place a widget


#run
# window.after(5000, getter)

#window.after(7000, window.destroy)
window.mainloop()



